In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms


class MyDataset(Dataset):
    def __init__(self, ):
        translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly",\
  "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep",\
"scoiattolo": "squirrel",  "cavallo": "horse",\
  "farfalla":"butterfly",
  "ragno":"spider", }
        path="/kaggle/input/animals10/raw-img"
        kk=[]
        labels=[]
        for root,dir,name in  os.walk(path):
            if(len(dir)==0):
                label=root.split("/")[-1]
                label=translate[label]
                for na in name:
                    tp_path=os.path.join(root,na)
                    kk.append(tp_path)
                    labels.append(label)

        self.path= kk
        self.labels = labels
        
    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        tp_path = self.path[idx]
        image=Image.open(tp_path)
        transform_to_tensor = transforms.ToTensor()

        image = transform_to_tensor(image)
        label = self.labels[idx]
        file_name=tp_path
        return image, label,file_name
my_dataset = MyDataset()
dataloader = DataLoader(my_dataset, batch_size=1, shuffle=True)
"""
for i,(image, label) in enumerate(dataloader):
    if(i<5):
        transform_to_pil = transforms.ToPILImage()
        image=image.squeeze(0)
        image = transform_to_pil(image)
        display(image)
        print(label)
    
"""

'\nfor i,(image, label) in enumerate(dataloader):\n    if(i<5):\n        transform_to_pil = transforms.ToPILImage()\n        image=image.squeeze(0)\n        image = transform_to_pil(image)\n        display(image)\n        print(label)\n    \n'

In [2]:

from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import pandas as pd
device="cuda"
# 加载预训练的BLIP模型和处理器
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")

transform_to_pil = transforms.ToPILImage()

describ=[]
colum=[]
labels=[]
for i,(image, label,file_name) in enumerate(dataloader):
    

   
    
    image=image.squeeze(0)
    image = transform_to_pil(image)

    # 处理图像
    inputs = processor(images=image, return_tensors="pt").to(device)

    # 生成输出
    outputs = model.generate(**inputs, max_length=200, num_beams=5)


    story = processor.decode(outputs[0], skip_special_tokens=True)
    if(i%200==9):
        print(story)
        
    describ.append(story)
    colum.append(file_name[0])
    labels.append(label[0])
        
        
    
data={
    "file_name":colum,
    "labels":labels,
    "desciption":describ
}
df=pd.DataFrame(data)
df.to_csv('output.csv', index=False)

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

2024-06-30 12:55:44.217372: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 12:55:44.217514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 12:55:44.338782: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

there is a small black and white dog sitting on a person's lap
this is a close up of a gray cat with yellow eyes looking at the camera
there is a spider that is sitting on top of a piece of paper
there is a spider that is sitting on top of a flower
there are two dogs that are kissing each other on the pink carpet
there are two elephants standing next to each other near a tree
there is a man sitting on the ground with an umbrella next to an elephant
there is a white chicken sitting on top of a wooden crate
portrait of a black dog on a black background
there is a horse that is standing in the grass with a red rope
there is a squirrel that is climbing up the side of a tree
there is a bug that is sitting on top of a leaf
there are two small white dogs that are sitting on a table
there are two chickens standing next to each other in the grass
there is a cat that is running in the grass with it's tail in the air
there is a white cow that is laying down in the dirt
this is a close up of a spi